In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import gym
import numpy as np

# Define Policy Network
class Policy(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

def discount_returns(rewards, gamma=0.99):
    discounted_returns = []
    running_add = 0
    for t, r in enumerate(reversed(rewards)):
        # print(t)
        running_add = running_add * gamma ** t + r
        discounted_returns.insert(0, running_add)
    return discounted_returns

# Initialize the environment
env = gym.make('CartPole-v1')
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n

# Initialize the policy network
policy = Policy(input_dim, output_dim)

# Initialize the optimizer
optimizer = optim.Adam(policy.parameters(), lr=0.01)

# Training loop
for episode in range(1):
    state = env.reset()
    episode_reward = 0

    states = []
    actions = []
    rewards = []

    while True:
        # env.render()  # Render the environment

        states.append(torch.tensor(state, dtype=torch.float32))

        action_probs = policy(torch.tensor(state, dtype=torch.float32))

        # take the action based on the probability. Higher probability will have a higher chance, but not always
        action = torch.multinomial(action_probs, num_samples=1).item() 
        
        actions.append(action)

        # print("Action Probs: {}, Action: {}".format(action_probs, action))
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)

        episode_reward += reward
        state = next_state

        if done:
            # Compute discounted rewards
            discounted_returns = torch.tensor(discount_returns(rewards), dtype=torch.float32)
            print("Discounted Returns: {}".format(discounted_returns))

            # Compute policy loss, resulting the log value of the choosen probability of each state that choosen by action in each timestep 
            log_probs = torch.log(policy(torch.stack(states)))[range(len(actions)), actions]
            
            # Compute policy loss
            policy_loss = -(log_probs * discounted_returns).mean()

            # print("States: {}".format(states))
            # print("Stacked States: {}".format(torch.stack(states)))
            # print("Policy: {}".format(policy(torch.stack(states))))
            # print("Log: {}".format(torch.log(policy(torch.stack(states)))))
            # print("Actions: {}".format(actions))
            # print("Len Actions: {}".format(len(actions)))
            # print("Range Actions: {}".format([range(len(actions)), actions]))
            print("Log Probs: {}".format(log_probs))
            # print("States: {}, Actions: {}".format(policy(torch.stack(states)), [range(len(actions)), actions]))
            
            print("Policy Loss: {}".format(policy_loss))

            # Update policy
            optimizer.zero_grad()
            policy_loss.backward()
            optimizer.step()

            print("Episode: {}, Reward: {}".format(episode, episode_reward))
            break

env.close()


/home/ardie85/PHD/Research/code/.venv/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn


Discounted Returns: tensor([2.8511, 2.9097, 2.9719, 3.0378, 3.1080, 3.1830, 3.2632, 3.3493, 3.4419,
        3.5419, 3.6499, 3.7671, 3.8943, 4.0325, 4.1829, 4.3464, 4.5240, 4.7165,
        4.9244, 5.1478, 5.3864, 5.6393, 5.9049, 6.1804, 6.4624, 6.7459, 7.0252,
        7.2929, 7.5408, 7.7595, 7.9387, 8.0677, 8.1356, 8.1315, 8.0456, 7.8692,
        7.5955, 7.2199, 6.7406, 6.1590, 5.4797, 4.7106, 3.8628, 2.9504, 1.9900,
        1.0000])
Log Probs: tensor([-0.6385, -0.6410, -0.6538, -0.7130, -0.6537, -0.7131, -0.7332, -0.6439,
        -0.6562, -0.7101, -0.6577, -0.7085, -0.7283, -0.7374, -0.6452, -0.6528,
        -0.7229, -0.7337, -0.7385, -0.7415, -0.7425, -0.7430, -0.6446, -0.6438,
        -0.7435, -0.6425, -0.7430, -0.7474, -0.7477, -0.6426, -0.6400, -0.7482,
        -0.7519, -0.6402, -0.6359, -0.7503, -0.6342, -0.6383, -0.6377, -0.6405,
        -0.7397, -0.7497, -0.6353, -0.6405, -0.6530, -0.6747],
       grad_fn=<IndexBackward0>)
Policy Loss: 3.5534894466400146
Episode: 0, Reward: 46.0